In [ ]:
%pip install git+https://github.com/jvkersch/pyconcorde

In [ ]:
# install tweak file
# Prom knows this

%cp '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Concorde/concorde_class.py' '/content'
%cp '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Concorde/test.py' '/content'
%cp '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Concorde/concorde_to_replace/tsp.py' '/usr/local/lib/python3.10/dist-packages/concorde'
%cp '/content/drive/Shareddrives/Private Unlimited Drive #1/DDS/Analytics Project/Concorde/concorde_to_replace/util.py' '/usr/local/lib/python3.10/dist-packages/concorde'

In [ ]:
import os
import pandas as pd
import numpy as np
import re
import collections
import pickle
from ast import literal_eval
import random
import scipy
import math
from concorde_class import Concorde
import datetime
import subprocess

#Functions

In [ ]:
# heuristic solution with LKH solver
# no mapping yet

class lkh:
    def __init__(self , dist_mat_pic_file):
        self.dist_mat_pic_file = dist_mat_pic_file
        self.dist_mat_file_name = dist_mat_pic_file.split(".")[0]
        print(self.dist_mat_file_name)

    def solve(self):
        '''
        solves the TSP instance using .exe file
        '''
        # Generate the .par file
        self.generate_parameter_file(self.dist_mat_file_name)

        # Generate the .atsp file
        dist_mat = self.read_pickle(self.dist_mat_pic_file)
        self.generate_atsp_file(dist_mat , self.dist_mat_file_name)

        try :    
            # Replace "E3k.0.par" with the actual parameter for your .exe file
            result = subprocess.check_output(["LKH-2.exe", f"{self.dist_mat_file_name}.par", "-S" , f"{self.dist_mat_file_name}_start.txt" ], stdin=subprocess.DEVNULL , timeout = 300)
            #result = subprocess.check_output(["LKH-2.exe", f"{self.dist_mat_file_name}.par", "-S" , f"{self.dist_mat_file_name}_start.txt" ], stdin=subprocess.DEVNULL , timeout = 300)
        
        except subprocess.TimeoutExpired:
            print("Execution timed out. Terminating the process.")
            # print(output)
        
        result_file_name = self.dist_mat_file_name + '.txt'
        print("Result is saved in : ",result_file_name)
        length , tour_section = self.get_obj_val_and_tour(result_file_name)
        length_rescaled = length/100

        print("Objective function value : ", length_rescaled)
        print("The routing : " , tour_section)

        return length_rescaled, tour_section

    
    def get_obj_val_and_tour(self , tour_file):
        """
        Extracts the length and tour section from a tour file.

        Args:
            tour_file (str): Path to the tour file.

        Returns:
            tuple: A tuple containing the length and tour section as a list.
        """
        length = None
        tour_section = []

        with open(tour_file, "r") as file:
            for line in file:
                line = line.strip()

                if line.startswith("COMMENT : Length"):
                    length = int(line.split("=")[1].strip())
                elif line == "TOUR_SECTION":
                    # Start reading the tour section
                    for tour_line in file:
                        tour_line = tour_line.strip()
                        if tour_line == "-1":
                            # End of tour section
                            break
                        tour_section.append(int(tour_line))

        return length, tour_section


    def get_obj_val(self , output):
        '''
        returns the objective value from the  long output value
        '''
        # Assuming `output` contains the captured output
        cost_min_match = re.search(r'Cost\.min = (\d+)', output)

        if cost_min_match:
            cost_min_value = int(cost_min_match.group(1))
            return cost_min_value
        else:
            print("Cost.min not found in the output.")
            return 0    
    
    def get_routing(self , filename):
        '''
        returns the routing in a numpy array
        '''
        start_keyword = 'TOUR_SECTION'
        end_keyword = '-1'
        with open(filename, 'r') as file:
            lines = []
            read_flag = False
            for line in file:
                if start_keyword in line:
                    read_flag = True
                    continue  # skip this line
                if read_flag:
                        if end_keyword in line:
                            read_flag = False
                            break
                        else:
                            lines.append(line.strip())

        lines = np.array(lines)
        return lines
    
    def generate_parameter_file(self , filename):
        runs = 1
        filename_atsp = filename + ".atsp"
        filename_txt = filename + ".txt"  
        filename_par = filename + ".par"

        with open(filename_par, 'w') as f:
            f.write(f"PROBLEM_FILE = {filename_atsp}\n")
            f.write(f"OUTPUT_TOUR_FILE = {filename_txt}\n")
            f.write(f"OPTIMUM = 40634081 \n")
            f.write(f"INITIAL_PERIOD = 1000 \n")
            f.write(f"MAX_CANDIDATES = 6 \n")
            f.write(f"MAX_TRIALS = 1000 \n")
            f.write(f"MOVE_TYPE = 6 \n")
            f.write(f"PATCHING_C = 6 \n")
            f.write(f"PATCHING_A = 5 \n")
            f.write(f"RECOMBINATION = GPX2 \n")
            f.write(f"RUNS = {runs} \n")

    
    def multiply_round_up(self , lst):
        '''
        input : list of lists having non integral values
        output : list of lists having integral values.
        Function to multiply, round up numbers,  and return as integers
        '''
        rounded_lst = np.rint(np.array(lst)*100).astype(int)
        rounded_lst = rounded_lst.tolist()
        return rounded_lst


    def read_pickle(self,pickle_path):
        # Open the pickle file in read-binary mode
        with open(pickle_path, 'rb') as f:
            # Load the object from the pickle file
            pic_obj = pickle.load(f)
        pic_obj = self.multiply_round_up(pic_obj)
        return pic_obj
    
    def generate_atsp_file(self , dist_mat, filename):   
        '''
        input : 
        a symmentric matrix  
        directory of a .tsp file you want to generate
        number of cities in the TSP

        Output : 
        a .tsp file generated in the filename directory
        '''
        # Defining the number of cities
        num_cities = len(dist_mat)

        # Write the coordinates to a .atsp file
        # fname = filename.split("/")[-1].split(".")[0]
        fname = filename 
        with open(filename+".atsp", 'w') as f:
            f.write(f"NAME: {fname}\n")
            # f.write("COMMENT: Randomly generated TSP instance\n")
            f.write("TYPE: TSP (M.~Hofmeister)\n")
            f.write(f"DIMENSION: {num_cities}\n") #becase the distance matrix doubles in size when converted to a symmentric instance
            f.write("EDGE_WEIGHT_TYPE: EXPLICIT\n")
            f.write("EDGE_WEIGHT_FORMAT: FULL_MATRIX\n")
            # f.write("NODE_COORD_SECTION\n")
            f.write("DISPLAY_DATA_TYPE: NO_DISPLAY\n")
            f.write("EDGE_WEIGHT_SECTION\n")
            for i in range(len(dist_mat)):
                for j in range(len(dist_mat)):
                    f.write(f'{str(dist_mat[i][j])}\t')
                f.write("\n")
            f.write("EOF\n")

In [ ]:
# exact solution with concord
# no mapping yet

def concord_solver(distance_matrix):
  concorde = Concorde()
  concorde_sol = concorde.run_ProbATSP(distance_matrix)
  return concorde.route, concorde.optimal_value

# From Instances

In [ ]:
# folder that contains instance distance matrix files
# files from github : Pickle/Warmsen_Instances_50/dm

instances_path = aa

## LKH, Stopping Criteria 60 s

In [ ]:
i = 1
for filename in os.listdir(instances_path):
  start = datetime.datetime.now()
  dum = lkh(instances_path + '/' + filename)
  length , tour_section = dum.solve()
  print('time needed -->', datetime.datetime.now() - start)
  print('\n')
  i = i+1

## LKH, Stopping Criteria 30 s

In [ ]:
i = 1
for filename in os.listdir(instances_path):
  start = datetime.datetime.now()
  dum = lkh(instances_path + '/' + filename)
  length , tour_section = dum.solve()
  print('time needed -->', datetime.datetime.now() - start)
  print('\n')
  i = i+1

## LKH, Stopping Criteria 15 s

In [ ]:
i = 1
for filename in os.listdir(instances_path):
  start = datetime.datetime.now()
  dum = lkh(instances_path + '/' + filename)
  length , tour_section = dum.solve()
  print('time needed -->', datetime.datetime.now() - start)
  print('\n')
  i = i+1

##Concorde

In [ ]:
i = 1
for filename in os.listdir(instances_path):
  start = datetime.datetime.now()
  dm_instance = pickle.load(open(instances_path + '/' + filename, "rb"))
  dm_instance_scaled = np.rint(np.array(dm_instance)*100).astype(int)
  dm_instance_scaled = dm_instance_scaled.tolist()
  tour, opt_val = concord_solver(dm_instance_scaled)
  print(i, ':', filename, 'total cities ', len(dm_instance), ':', 'opt value ', opt_val/100)
  print('time needed -->', datetime.datetime.now() - start)
  print('\n')
  i = i+1

# From Original Distance Matrix of Districts

In [ ]:
# folder that contains original distance matrix files
# files from github : Pickle/Warmsen_original/dm

original_path = bb

## LKH, Stopping Criteria 60 s

In [ ]:
i = 1
for filename in os.listdir(original_path):
  start = datetime.datetime.now()
  dum = lkh(original_path + '/' + filename)
  length , tour_section = dum.solve()
  print('time needed -->', datetime.datetime.now() - start)
  print('\n')
  i = i+1

## LKH, Stopping Criteria 30 s

In [ ]:
i = 1
for filename in os.listdir(original_path):
  start = datetime.datetime.now()
  dum = lkh(original_path + '/' + filename)
  length , tour_section = dum.solve()
  print('time needed -->', datetime.datetime.now() - start)
  print('\n')
  i = i+1

## LKH, Stopping Criteria 15 s

In [ ]:
i = 1
for filename in os.listdir(original_path):
  start = datetime.datetime.now()
  dum = lkh(original_path + '/' + filename)
  length , tour_section = dum.solve()
  print('time needed -->', datetime.datetime.now() - start)
  print('\n')
  i = i+1

##Concorde

In [ ]:
i = 1
for filename in os.listdir(original_path):
  start = datetime.datetime.now()
  dm_instance = pickle.load(open(original_path + '/' + filename, "rb"))
  dm_instance_scaled = np.rint(np.array(dm_instance)*100).astype(int)
  dm_instance_scaled = dm_instance_scaled.tolist()
  tour, opt_val = concord_solver(dm_instance_scaled)
  print(i, ':', filename, 'total cities ', len(dm_instance), ':', 'opt value ', opt_val/100)
  print('time needed -->', datetime.datetime.now() - start)
  print('\n')
  i = i+1